# Project: Feature Evaluation

### Получаем данные за май и июнь из БД, сводим в одну таблицу данные по когортам пользователей и добавляем информацию о прибыли. 

In [159]:
%matplotlib inline
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras 

def getRegisteredUsersCount():
    query = '''SELECT r.registration_date, count(distinct(r.user_id)) as registered_users_count
    FROM case9.registration r
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

registered_users_count['registration_date'] = pd.to_datetime(registered_users_count['registration_date'])

def getActiveUsersCountWithCohorts():
    query = '''SELECT ac.activity_date, r.registration_date, count(distinct(ac.user_id)) as active_users_count
    FROM case9.user_activity ac
    LEFT JOIN case9.registration r ON r.user_id = ac.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

active_users_count_with_cohorts = pd.DataFrame(getActiveUsersCountWithCohorts())

for col in ['activity_date','registration_date']:
    active_users_count_with_cohorts[col] = pd.to_datetime(active_users_count_with_cohorts[col])

retention_table = active_users_count_with_cohorts.merge(registered_users_count,on=['registration_date'],how='left')

retention_table['retention_rate'] = retention_table['active_users_count'] / retention_table['registered_users_count']
retention_table['lifetime'] = retention_table['activity_date'] - retention_table['registration_date']
retention_table['lifetime'] = retention_table['lifetime']/np.timedelta64(1,'D')
retention_table['lifetime'] = retention_table['lifetime'].astype(int) # Приведем тип к целому числу
retention_table.head()

,activity_date,registration_date,active_users_count,registered_users_count,retention_rate,lifetime
0,2019-05-01,2019-05-01,2590,6516,0.397483,0
1,2019-05-02,2019-05-01,2146,6516,0.329343,1
2,2019-05-02,2019-05-02,2049,5309,0.385948,0
3,2019-05-03,2019-05-01,1637,6516,0.251228,2
4,2019-05-03,2019-05-02,1817,5309,0.342249,1


In [160]:
def getRevenue():
    query = '''SELECT re.date, r.registration_date, count(distinct(re.user_id)) as users_count_with_revenue, sum(re.revenue) as revenue
    FROM case9.user_revenue re
    LEFT JOIN case9.registration r ON r.user_id = re.user_id
    WHERE r.registration_date >= '2019-05-01' and r.registration_date < '2019-07-01'
    GROUP BY 1,2
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='84.201.134.129' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

revenue = pd.DataFrame(getRevenue())

for col in ['date','registration_date']:
    revenue[col] = pd.to_datetime(revenue[col])

revenue = revenue.rename(columns={'date':'activity_date'})

revenue.head()

,activity_date,registration_date,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,1672,2357.580
1,2019-05-02,2019-05-01,1420,514.679
2,2019-05-02,2019-05-02,1367,1629.140
3,2019-05-03,2019-05-01,1048,390.952
4,2019-05-03,2019-05-02,1164,533.608


In [161]:
retention_table_with_revenue = retention_table.merge(revenue,on=['registration_date','activity_date'],how='left')
for col in ['revenue','users_count_with_revenue']:
    retention_table_with_revenue[col] = retention_table_with_revenue[col].fillna(0)
retention_table_with_revenue['users_count_with_revenue'] = retention_table_with_revenue['users_count_with_revenue'].astype(int)
retention_table_with_revenue.head()

,activity_date,registration_date,active_users_count,registered_users_count,retention_rate,lifetime,users_count_with_revenue,revenue
0,2019-05-01,2019-05-01,2590,6516,0.397483,0,1672,2357.580
1,2019-05-02,2019-05-01,2146,6516,0.329343,1,1420,514.679
2,2019-05-02,2019-05-02,2049,5309,0.385948,0,1367,1629.140
3,2019-05-03,2019-05-01,1637,6516,0.251228,2,1048,390.952
4,2019-05-03,2019-05-02,1817,5309,0.342249,1,1164,533.608


### Определяем Retention 7 дня для периодов май и июнь
Для этого отфильтруем созданную таблицу по датам и проведем расчеты возвращаемости.

In [162]:
retention_7_may=retention_table_with_revenue[(retention_table_with_revenue['lifetime']==7)& (retention_table_with_revenue['registration_date']>='2019-05-01') & (retention_table_with_revenue['registration_date']<'2019-06-01')]
retention_level_7_may=retention_7_may['active_users_count'].sum()/retention_7_may['registered_users_count'].sum()

retention_7_june=retention_table_with_revenue[(retention_table_with_revenue['lifetime']==7)& (retention_table_with_revenue['registration_date']>='2019-06-01')]
retention_level_7_june=retention_7_june['active_users_count'].sum()/retention_7_june['registered_users_count'].sum()

print("Retention rate on 7th day for May is {0:2.2f} %".format(retention_level_7_may*100))
print("Retention rate on 7th day for June is {0:2.2f} %".format(retention_level_7_june*100))

Retention rate on 7th day for May is 12.93 %
Retention rate on 7th day for June is 12.56 %


#### Как видно из расчетов, в июне retention rate немного снизился по сравнению с маем.

### Определяем ARPPU для мая и июня

In [185]:
revenue_may = retention_table_with_revenue['revenue'][(retention_table_with_revenue['registration_date']>='2019-05-01') & (retention_table_with_revenue['registration_date']<'2019-06-01')].sum() 
paying_users_may= retention_table_with_revenue['users_count_with_revenue'][(retention_table_with_revenue['registration_date']>='2019-05-01') & (retention_table_with_revenue['registration_date']<'2019-06-01')].sum()

revenue_june = retention_table_with_revenue['revenue'][(retention_table_with_revenue['registration_date']>='2019-06-01')].sum() 
paying_users_june= retention_table_with_revenue['users_count_with_revenue'][retention_table_with_revenue['registration_date']>='2019-06-01'].sum()
print("With given number of {0} registered paying users in May, revenue of {1:2.2f} units, ARPPU = {2:2.3f} units".format(paying_users_may,revenue_may,revenue_may/paying_users_may))
print("With given number of {0} registered paying users in June, revenue of {1:2.2f} units, ARPPU = {2:2.3f} units".format(paying_users_june,revenue_june,revenue_june/paying_users_june))
print("Percent change of paying users June vs May {0:2.2f}".format(((paying_users_june/paying_users_may)-1)*100))
print("Percent change of revenue June vs May {0:2.2f}".format(((revenue_june/revenue_may)-1)*100))
print("Percent change of ARPPU June vs May {0:2.2f}%".format(((revenue_june/paying_users_june)/(revenue_may/paying_users_may)-1)*100))

With given number of 398525 registered paying users in May, revenue of 121998.30 units, ARPPU = 0.306 units
With given number of 202663 registered paying users in June, revenue of 79685.87 units, ARPPU = 0.393 units
Percent change of paying users June vs May -49.15
Percent change of revenue June vs May -34.68
Percent change of ARPPU June vs May 28.44%


## Комментарии:
#### Возвращаемость клиентов в июне снизилась по сравнению с маем. Так же снизилась и абсолютная прибыль, но так как количество пользователей уменьшилось значительней, чем прибыль, то наш ARPPU вырос на 28.44%
#### При данных результатах оценки невозможно сказать, действительно ли новая фича полезна. Следует либо понаблюдать еще месяц, либо провести более глубокий анализ по другим метрикам.